In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Malicious Macro Detection
%ls

/content/drive/MyDrive/Colab Notebooks/Malicious Macro Detection
 AdaBoostClassifier.joblib
 classifiers_recall_scores.joblib
 CNNClassifier.joblib
 DecisionTreeClassifier.joblib
 EDA.ipynb
 features_k_1000.joblib
 features_k_100.joblib
 features_k_10.joblib
 features_k_1200.joblib
 features_k_1500.joblib
 features_k_2000.joblib
 features_k_2500.joblib
 features_k_3000.joblib
 features_k_500.joblib
 features_k_50.joblib
 gnbClassifier.joblib
 GradientBoostingClassifier.joblib
 knnClassifier.joblib
 LSTMClassifier.joblib
'Macro Malware Detection using Machine Learning Techniques A New Approach '
 mlpClasifier.joblib
 __pycache__/
 randomForestClassifier.joblib
 recall_scores.joblib
 RobertaClassifier.joblib
 svmClassifier.joblib
 test_dataset.csv
 test_loader.joblib
 test_loader.pkl
 tfidf_1000.joblib
 tfidf_100.joblib
 tfidf_10.joblib
 tfidf_1200.joblib
 tfidf_1500.joblib
 tfidf_2000.joblib
 tfidf_2500.joblib
 tfidf_3000.joblib
 tfidf_500.joblib
 tfidf_50.joblib
 train_dataset.csv
 tra

In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from utils import save_loader

from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from sklearn.metrics import make_scorer, recall_score
from sklearn.model_selection import GridSearchCV
from joblib import load

from vba_pipeline import create_pipeline, evaluate_model

In [ ]:
train_set = pd.read_csv('train_dataset.csv', encoding='utf-16le')
val_set = pd.read_csv('validation_dataset.csv', encoding='utf-16le')
test_set = pd.read_csv('test_dataset.csv', encoding='utf-16le')

In [ ]:
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_features': ['auto', 'sqrt', 'log2'],
    'model__max_depth': [4, 6, 8, 10],
    'model__criterion': ['gini', 'entropy']
}

mapper = {
    'white' : 1,
    'mal' : 0
}

In [ ]:
x_train, y_train = train_set['vba_code'], train_set['label']
x_val, y_val = val_set['vba_code'], val_set['label']
x_test, y_test = test_set['vba_code'], test_set['label']

y_train = y_train.map(mapper)
y_val = y_val.map(mapper)
y_test = y_test.map(mapper)

In [ ]:
rf_pipeline = create_pipeline(model=RandomForestClassifier(), vectorizer=TfidfVectorizer())

In [ ]:
scorer = make_scorer(recall_score, average='binary')

grid_search_rf = GridSearchCV(estimator=rf_pipeline, param_grid=param_grid, scoring=scorer, cv=5)

grid_search_rf

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocess',
                                        <vba_pipeline.PreprocessTransformer object at 0x7d367b6dba90>),
                                       ('vectorizer', TfidfVectorizer()),
                                       ('scaler',
                                        StandardScaler(with_mean=False)),
                                       ('model', RandomForestClassifier())]),
             param_grid={'model__criterion': ['gini', 'entropy'],
                         'model__max_depth': [4, 6, 8, 10],
                         'model__max_features': ['auto', 'sqrt', 'log2'],
                         'model__n_estimators': [100, 200, 300]},
             scoring=make_scorer(recall_score, response_method='predict', average=binary))

In [ ]:
grid_search_rf.fit(x_train, y_train)

print("Best parameters found: ", grid_search_rf.best_params_)
print("Best recall score: ", grid_search_rf.best_score_)

/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py:540: FitFailedWarning: 
120 fits failed out of a total of 360.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
120 fits failed with the following error:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/base.py", line 1473, in wrapper
    return fit_method(estimator, *args, **kwargs)
  File "/usr/local/lib/python3.10/dist-packages/sklearn/pipeline.py", line 473, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/usr/local/lib/p

Best parameters found:  {'model__criterion': 'gini', 'model__max_depth': 4, 'model__max_features': 'sqrt', 'model__n_estimators': 100}
Best recall score:  1.0


In [ ]:
best_params = grid_search_rf.best_params_
rf_params = {k.replace('model__', ''): v for k, v in best_params.items()}
rf_model = RandomForestClassifier(**rf_params)

In [ ]:
print(best_params)

{'model__criterion': 'gini', 'model__max_depth': 4, 'model__max_features': 'sqrt', 'model__n_estimators': 100}


In [ ]:
rf_model = load('randomForestClassifier.joblib')
x_train_1000 = load('x_train_1000.joblib')

In [ ]:
rf_model.fit(x_train_1000, y_train)

RandomForestClassifier(max_depth=4)

In [ ]:
save_loader('/content/drive/MyDrive/Colab Notebooks/Malicious Macro Detection/randomForestClassifier.joblib' ,rf_model)

RandomForestClassifier(max_depth=4) saved sucessfuly
